# Decoding Quantum CSP Codes

Here, we will decode the Quantum CSP codes as discussed in https://arxiv.org/abs/2207.03562

In [ ]:
import os
import sys
import json

sys.path.append("../")
sys.path.append("../..")

import numpy as np
import qecstruct as qc
import qecsim.paulitools as pt
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import FuncFormatter, FormatStrFormatter
from tqdm import tqdm
from scipy.stats import sem

from mdopt.mps.utils import create_custom_product_state
from mdopt.contractor.contractor import mps_mpo_contract
from mdopt.optimiser.utils import (
    SWAP,
    COPY_LEFT,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)

from examples.decoding.decoding import (
    css_code_checks,
    css_code_logicals,
    css_code_logicals_sites,
    css_code_constraint_sites,
    apply_constraints,
    apply_bitflip_bias,
    apply_depolarising_bias,
    decode_custom,
    pauli_to_mps,
    decode_css,
    css_code_stabilisers,
    multiply_pauli_strings,
    map_distribution_to_pauli,
    generate_pauli_error_string,
)
from examples.decoding.visualisation import plot_parity_check_mpo

In [ ]:
NUM_EXPERIMENTS = 10

SEED = 123
seed_seq = np.random.SeedSequence(SEED)

max_bond_dims = [16, 8, 4]
error_rates = np.linspace(1e-2, 0.2, 10)

qubit_sizes = [30, 40, 50, 60]  # , 70, 80, 90, 100]

for batch in range(1, 15):
    for num_qubits in qubit_sizes:
        code_path = f"data-csp-codes/batch_{batch}/codes/qubits_{num_qubits}"

        for code in os.listdir(code_path):
            if code.endswith(".json"):
                code_file = os.path.join(code_path, code)

                with open(code_file, "r") as code_json:
                    code_data = json.load(code_json)
                    x_code = LinearCode(
                        BinaryMatrix(
                            num_columns=code_data["num_qubits"],
                            rows=code_data["x_stabs"],
                        )
                    )
                    z_code = LinearCode(
                        BinaryMatrix(
                            num_columns=code_data["num_qubits"],
                            rows=code_data["z_stabs"],
                        )
                    )
                    quantum_csp_code = qc.CssCode(x_code=x_code, z_code=z_code)

                    for CHI_MAX in max_bond_dims:
                        print(
                            f"Processing: Batch {batch}, Qubits {num_qubits}, Code {code}, CHI_MAX = {CHI_MAX}"
                        )
                        for ERROR_RATE in tqdm(error_rates):
                            failures = []

                            for l in range(NUM_EXPERIMENTS):
                                new_seed = seed_seq.spawn(1)[0]
                                rng = np.random.default_rng(new_seed)
                                random_integer = rng.integers(1, 10**8 + 1)
                                SEED = random_integer

                                error = generate_pauli_error_string(
                                    len(quantum_csp_code),
                                    ERROR_RATE,
                                    seed=SEED,
                                    error_model="Depolarising",
                                )
                                error = pauli_to_mps(error)

                                _, success = decode_css(
                                    code=quantum_csp_code,
                                    error=error,
                                    chi_max=CHI_MAX,
                                    bias_type="Depolarising",
                                    bias_prob=ERROR_RATE,
                                    renormalise=True,
                                    silent=True,
                                    contraction_strategy="Optimised",
                                )

                                failures.append(1 - success)

                            # Store results in a structured filename
                            result_filename = (
                                f"numqubits{num_qubits}_bonddim{CHI_MAX}_errorrate{ERROR_RATE:.12f}_"
                                f"seed{SEED}_batch{batch}_{os.path.splitext(code)[0]}.npy"
                            )
                            np.save(result_filename, np.array(failures))

[[0, 1, 5, 6, 7, 9, 13, 20, 23, 24, 25, 27], [2, 5, 6, 9, 13, 18, 20, 28], [10, 14, 15, 16, 19, 21, 26, 28], [2, 4, 10, 13, 15, 17, 18, 25, 26, 27], [9, 10, 20, 21, 22, 23, 26], [1, 2, 3, 8, 11, 12, 16, 17, 19, 22, 24, 27], [1, 9, 14, 19, 24, 25], [0, 1, 2, 5, 7, 15, 23], [7, 14, 16, 23, 25, 26, 27, 29], [4, 13, 16, 17, 25, 27, 28], [6, 7, 11, 12, 13, 18, 19, 21, 22, 29], [3, 8, 11, 12, 25, 26], [0, 3, 4, 8, 27, 29]]
[[1, 6, 7, 8, 11, 15, 18, 21, 25, 28, 34], [1, 6, 9, 15, 28, 29, 34], [0, 3, 6, 7, 11, 17, 21, 23, 24, 27, 28, 29, 32, 34, 36, 38, 39], [0, 6, 16, 26, 27, 33, 35], [4, 13, 14, 20, 24, 33, 35, 37], [2, 3, 4, 7, 8, 9, 10, 12, 13, 17, 22, 26, 27, 31, 32, 33, 35, 36, 38, 39], [5, 7, 21, 27, 29, 31], [1, 5, 24, 27, 36, 39], [3, 5, 11, 16, 27, 30], [7, 17, 18, 24, 27, 36], [10, 11, 25, 29, 37, 38], [4, 14, 19, 22, 23, 32], [2, 6, 7, 13, 15, 26], [12, 16, 23, 30, 35, 36], [8, 13, 15, 22, 30, 31], [0, 7, 9, 10, 12, 14, 15, 35, 37], [2, 7, 13, 18, 19, 20, 25, 28, 35], [12, 13, 19, 